In [1]:
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import pandas as pd
# 모든 행을 출력하도록 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 기본값으로 설정 (처음 5개와 마지막 5개 행만 출력)
pd.reset_option('display.max_rows')

# 출력 포맷 설정 (소수점 4자리까지)
pd.options.display.float_format = '{:.4f}'.format

import platform
import seaborn as sns

import matplotlib.pyplot as plt

# 운영 체제 확인
if platform.system() == 'Darwin':  # Mac
    print('apple gothic')
    font_name = 'AppleGothic'
elif platform.system() == 'Windows':  # Windows
    font_name = 'NanumGothic'
else:
    font_name = None

# 한글 폰트 설정
if font_name:
    plt.rcParams['font.family'] = font_name

# 마이너스 부호 설정
plt.rcParams['axes.unicode_minus'] = False

apple gothic


## 데이터 로드

In [2]:
# df = pd.read_csv('../data/5_데이터셋생성_시가총액추가전.csv', dtype={'거래소코드' : 'object'})
# df = pd.read_csv('../data/5_데이터셋생성_재무.csv', dtype={'거래소코드' : 'object'}) # label + 재무
# df = pd.read_csv('../data/5_데이터셋생성_재무+거시.csv', dtype={'거래소코드' : 'object'}) # label + 재무 + 거시
df = pd.read_csv('../data/5_데이터셋생성_재무+거시+수익성_최종.csv', dtype={'거래소코드' : 'object'}) # label + 재무 + 거시 + 수익성
df.shape

(1385, 226)

In [3]:
df.head()

,거래소코드,회사명,회계년도,상장일,상장폐지일,결산년도,상장년도,market,자산,유형자산,유동자산,재고자산,자본,매출액,매출원가,판매비와 관리비(물류원가 등 포함),기타(영업)비용,당기제조원가,급여,퇴직급여,상품매출원가,제품매출원가,감가상각비,대손상각비,개발비상각,기타무형자산상각비,외환차익1,외화환산이익1,외환차손1,외화환산손실1,외환차익2,외화환산이익2,외환차손2,외환환산손실2,무형자산,건설중인자산,토지,자본잉여금,이익잉여금(결손금),미처분이익잉여금(결손금),기타임의적립금,(연차배당),(중간배당),배당금지급(-),단기차입금,매입채무 및 기타유동채무,매출채권 및 기타유동채권,비유동자산,사채,유동금융자산,유동부채,유동성장기부채,장기매입채무 및 기타비유동채무,장기매출채권 및 기타비유동채권,장기차입금,현금및현금성자산,영업활동으로 인한 현금흐름,유형자산의 증가,무형자산의 증가,연구개발비,종업원수,비유동부채,부채,유형자산의증가,유형자산의감소,(투자활동으로 인한 현금유출액),투자활동으로 인한 현금유입액,자본금,관계기업 등 지분관련 투자자산,영업손익,이자보상배율,년도차,자산_전기,자산증가율,유형자산_전기,유형자산증가율,유동자산_전기,유동자산증가율,재고자산_전기,재고자산증가율,자본_전기,자본증가율,매출액_전기,매출액증가율,자산평균,유형자산평균,무형자산_전기,무형자산평균,건설중인자산_전기,건설중인자산평균,토지_전기,토지평균,자본평균,총자산회전율,감가상각율,배당률,판관비체크,순외환손익대비매출액,매출원가대비매출액,당기총제조비용,재고조정중의고정비,고정비,총비용,변동비,변동비대비매출액,고정비대비매출액,인건비대비매출액,인건비대비영업총비용,1인당매출액,1인당인건비,재고자산평균,비유동자산_전기,비유동자산평균,매출채권 및 기타유동채권평균,장기매출채권 및 기타비유동채권평균,매입채무 및 기타유동채무평균,장기매입채무 및 기타비유동채무평균,주주이익(버핏),순운전자본(민식),유무형자산,영업현금흐름대비투하자본,ROTCE현금흐름대체,유형자산대비현금흐름,유무형자산대비현금흐름,매출대비고정자산,단기차입금_전기,단기차입금평균,장기차입금_전기,장기차입금평균,유동비율,현금비율,당좌비율,자기자본비율,비유동비율,부채비율,유동부채비율,비유동부채비율,비유동장기적합률,차입금의존도,매출채권대비매입채무,순운전자본,순운전자본대비총자본,차입금대비매출액,매출채권대비매입채무_before,영업활동현금흐름 대 총자산,영업활동현금흐름 대 매출액,투자안정성비율,영업활동현금흐름 대 투자활동현금지출,경영자산회전율분모,매출채권,매입채무,차입금,경영자산회전율분모_전기,매출채권_전기,매입채무_전기,차입금_전기,부채_전기,자본금_전기,영업활동현금흐름 대 총부채,자기자본회전율,자본금회전율,경영자산회전율,비유동자산회전율,유형자산회전율,재고자산회전율,매출채권회전율,매입채무회전율,차입금 대 매출액,label,시가총액,상장주식수,배당수익률,DATE,DGS10,DGS1,DGS6MO,T10Y2Y,WTI_oil,Dubai_oil,realGDP_usa,real_PCE,Core_PCE,CPI_sticky,Core_CPI,manuf_PMI(R),non_manuf_PMI(R),manuf_GB,non_manuf_GB,Equip_inv,TB_rtn(1y),TB_rtn(10y),IR_sm,DXY,CB_spread(AA-),CB_spread(BBB-),경제고통지수,경상수지,수입금액지수,수입물량지수,수출금액지수,수출물량지수,미국수입금액,중국수입금액,미국수출금액,중국수출금액,실업률,실업률_증감,소비자물가상승률,근원물가상승률,근원물가상승률_식품에너지제외,GDP_growth,CLI(경기선행),CCI(경기동행),NSI(=뉴스심리지수),기업실사BSI_실적,기업실사BSI_전망,year,month,매출액정상영업이익률,매출액순이익률,매출액총이익률,총자본순이익률,총자본사업이익률,총자본정상영업이익률,자기자본정상영업이익률,자기자본순이익률
0,000040,케이알모터스(주),2012/12,1976/05/25,NaN,2012,1976,KOSPI,119327267.0000,55810192.0000,61684604.0000,24763334.0000,60797171.0000,97926654.0000,92177389.0000,11948008.0000,0.0000,0.0000,2579833.0000,350950.0000,0.0000,0.0000,35985.0000,1248799.0000,0.0000,650714.0000,155998.0000,0.0000,470896.0000,1310924.0000,0.0000,0.0000,0.0000,0.0000,1118203.0000,0.0000,42542501.0000,3884892.0000,7235571.0000,12088682.0000,0.0000,0.0000,0.0000,0.0000,10401663.0000,28964045.0000,26201155.0000,57642663.0000,0.0000,100000.0000,43762954.0000,0.0000,407646.0000,342768.0000,0.0000,10236751.0000,-4565749.0000,937241.0000,42700.0000,0.0000,249.0000,14767142.0000,58530096.0000,937241.0000,21084.0000,992041.0000,464863.0000,59670690.0000,0.0000,-6198743.0000,-6.4700,1.0000,133337774.0000,-10.5075,57761033.0000,-3.3774,73022140.0000,-15.5262,24384487.0000,1.5536,68764654.0000,-11.5866,127720215.0000,-23.3272,126332520.5000,56785612.5000,1804381.0000,1461292.0000,0.0000,0.0000,42542501.0000,42542501.0000,64780912.5000,0.7752,0.0972,-0.0000,True,-13.6075,94.1290,14842806.0000,7635041.5537,11035931.0537,104125397.0000,93089465.9463,95.0604,11.2696,2.9928,24.5295,393280.0000,10361.0000,24573910.5000,60315635.0000,58979149.0000,26201155.0000,342768.0000,28964045.0000,407646.0000,-4858991.0000,51117833.5000,58246904.5000,0.0000,0.0000,0.0000,0.0000,0.0000,10304725.0000,10353194.0000,0.0000,0.0000,140.9516,23.3914,23.6199,50.9499,94.8114,96.2711,71.9819,24.2892,76.2829,8.7169,90.3725,17921650.0000,15.0189,10.5724,90.3725,0.0000,0.0000,0.0000,0.0000,119327267.0000,26543923.0000,29371691.0000,10401663.0000,133337774.0000,33214201.0000,35842471.0000,10304725.0000,64573121.0000,59670690.0000,0.0000,1.5117,1.6411,0.7752,1.6604,1.7245,3.9850,3.2774,3.0032,10.5724,0.0000,58954641.2260,119341379.0000,-0.0000,2012-12-01,1.7274,0.1675,0.1282,1.4629,94.2008,3.3083,2.2891,1.3673,1.8500,-139.7270,0.7093,50.7000,56.1000,호황,호황,-9.4000,3.1237,3.4437,5.2200,2.5615,0.6325,6.2021,4.5000,1.9324,-5.6000,-1.7000,-2.4000,-1.1000,-16.6000,-6.3000,-1.0000,2.7000,3.1000,-0.1000,1.4000,1.1000,1.3000,0.0198,-0.1000,-0.5000,1.8700,-14,-17,2012,

In [4]:
df.columns

Index(['거래소코드', '회사명', '회계년도', '상장일', '상장폐지일', '결산년도', '상장년도', 'market', '자산',
       '유형자산',
       ...
       'year', 'month', '매출액정상영업이익률', '매출액순이익률', '매출액총이익률', '총자본순이익률',
       '총자본사업이익률', '총자본정상영업이익률', '자기자본정상영업이익률', '자기자본순이익률'],
      dtype='object', length=226)

In [5]:
# for col in df.columns:
#     print(col)

cols_info = [
    '회사명', '회계년도', '상장일', '상장폐지일',
    '결산년도', '상장년도', '년도차', 'year', 'month'
]
cols_drop = ['판관비체크', '매출채권대비매입채무_before']
cols_labeling = ['이자보상배율']
cols_before = [col for col in df.columns if col.endswith('_전기')]
cols_mean = [col for col in df.columns if col.endswith('평균')]

cols_nonfeature = cols_info + cols_drop + cols_labeling + cols_before + cols_mean

# 범주형 변수
cols_cate = ['manuf_GB', 'non_manuf_GB', 'market']

## 원핫인코딩

In [6]:
df['market'].value_counts()

market
KOSDAQ    957
KOSPI     428
Name: count, dtype: int64

In [7]:
# 원핫인코딩
# KOSDAQ -> 0 / KOSPI -> 1
df['market'] = df['market'].map(lambda x: 0 if x == 'KOSDAQ' else 1)

In [8]:
df['market'].value_counts()

market
0    957
1    428
Name: count, dtype: int64

In [9]:
df['manuf_GB'].value_counts()

manuf_GB
호황    1301
불황      84
Name: count, dtype: int64

In [10]:
# 원핫인코딩
# 불황 -> 0 / 호황 -> 1
df['manuf_GB'] = df['manuf_GB'].map(lambda x: 0 if x == '불황' else 1)

In [11]:
df[['label', 'manuf_GB']].value_counts().sort_index()

label   manuf_GB
0.0000  0            45
        1           857
1.0000  0            39
        1           444
Name: count, dtype: int64

In [12]:
df.columns[df.columns.str.startswith('manuf')]

Index(['manuf_PMI(R)', 'manuf_GB'], dtype='object')

In [13]:
# non_manuf_GB
df[cols_cate[1]].value_counts()

non_manuf_GB
호황    1385
Name: count, dtype: int64

In [14]:
# 원핫인코딩
# 불황 -> 0 / 호황 -> 1
df['non_manuf_GB'] = df['non_manuf_GB'].map(lambda x: 0 if x == '불황' else 1)
df['non_manuf_GB'].value_counts()

non_manuf_GB
1    1385
Name: count, dtype: int64

## 파일로 저장

In [15]:
# 파일로 저장

df.to_csv('../data/6_데이터셋_불필요컬럼제거.csv', index=None)